# Install and Import PyTorch
Install PyTorch if not already installed and import the necessary libraries, including torch and torchvision.

In [ ]:
# Install PyTorch if not already installed
!pip install torch torchvision

# Import the necessary libraries
import torch
import torchvision
from torchvision import transforms, datasets

# Verify PyTorch installation
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━ 51.6/66.5 MB 662.9 kB/s eta 0:00:23

# Load a Wildlife Dataset
Use torchvision.datasets to load a wildlife dataset, such as CIFAR-10 or a custom dataset.

In [ ]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(
    root='./data',  # Directory to store the dataset
    train=True,  # Load the training set
    download=True,  # Download the dataset if not already present
    transform=transform  # Apply the transformations
)

test_dataset = datasets.CIFAR10(
    root='./data',  # Directory to store the dataset
    train=False,  # Load the test set
    download=True,  # Download the dataset if not already present
    transform=transform  # Apply the transformations
)

# Create data loaders for the training and test sets
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False
)

# Display the size of the datasets
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of test samples: {len(test_dataset)}")

# Explore the Dataset
Visualize some sample images and their labels from the dataset to understand the data.

In [ ]:
# Visualize some sample images and their labels from the dataset
import matplotlib.pyplot as plt
import numpy as np

# Define a function to unnormalize and display images
def imshow(img):
    img = img / 2 + 0.5  # Unnormalize the image
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Convert from Tensor image to numpy image
    plt.show()

# Get a batch of training data
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Display the images
imshow(torchvision.utils.make_grid(images))

# Display the corresponding labels
classes = train_dataset.classes  # Get the class names
print(' '.join(f'{classes[labels[j]]}' for j in range(len(labels))))

# Define a Neural Network
Create a simple neural network using torch.nn.Module for wildlife classification.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Define a simple neural network for wildlife classification
class WildlifeNet(nn.Module):
    def __init__(self):
        super(WildlifeNet, self).__init__()
        # Define a convolutional layer
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        # Define a fully connected layer
        self.fc1 = nn.Linear(32 * 32 * 32, 10)  # CIFAR-10 has 10 classes

    def forward(self, x):
        # Apply the convolutional layer followed by ReLU activation
        x = F.relu(self.conv1(x))
        # Flatten the tensor for the fully connected layer
        x = x.view(-1, 32 * 32 * 32)
        # Apply the fully connected layer
        x = self.fc1(x)
        return x

# Instantiate the neural network
model = WildlifeNet()
print(model)  # Display the model architecture

# Train the Model
Write a training loop to train the model on the dataset using an optimizer and loss function.

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Stochastic Gradient Descent

# Training loop
num_epochs = 5  # Number of epochs to train the model
for epoch in range(num_epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get the inputs and labels
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 200:.3f}")
            running_loss = 0.0

print("Finished Training")

# Evaluate the Model
Evaluate the model's performance on a validation or test set and calculate metrics like accuracy.

In [ ]:
# Evaluate the model's performance on the test set
correct = 0
total = 0

# Disable gradient computation for evaluation
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
        total += labels.size(0)  # Total number of samples
        correct += (predicted == labels).sum().item()  # Count correct predictions

# Calculate and print accuracy
accuracy = 100 * correct / total
print(f"Accuracy of the model on the test set: {accuracy:.2f}%")

# Make Predictions
Use the trained model to make predictions on new wildlife images and visualize the results.

In [ ]:
# Make Predictions

import torchvision.transforms as T

# Define a function to make predictions on a single image
def predict_image(image, model, classes):
    # Apply the same transformations as the training set
    transform = T.Compose([
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    image = transform(image).unsqueeze(0)  # Add batch dimension
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
    return classes[predicted.item()]

# Load a sample image from the test dataset
from PIL import Image

sample_image_path = './data/cifar-10-batches-py/test_image.jpg'  # Replace with your image path
sample_image = Image.open(sample_image_path).convert('RGB')  # Ensure the image is in RGB format

# Make a prediction
predicted_class = predict_image(sample_image, model, classes)

# Visualize the image and the prediction
plt.imshow(sample_image)
plt.title(f"Predicted: {predicted_class}")
plt.axis('off')
plt.show()